In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
!unzip ../images.zip

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: images/TELEPHONIE - GPS/TELEPHONIE - GPS_6848.jpg  
  inflating: images/TELEPHONIE - GPS/TELEPHONIE - GPS_6849.jpg  
  inflating: images/TELEPHONIE - GPS/TELEPHONIE - GPS_685.jpg  
  inflating: images/TELEPHONIE - GPS/TELEPHONIE - GPS_6850.jpg  
  inflating: images/TELEPHONIE - GPS/TELEPHONIE - GPS_6851.jpg  
  inflating: images/TELEPHONIE - GPS/TELEPHONIE - GPS_6852.jpg  
  inflating: images/TELEPHONIE - GPS/TELEPHONIE - GPS_6853.jpg  
  inflating: images/TELEPHONIE - GPS/TELEPHONIE - GPS_6854.jpg  
  inflating: images/TELEPHONIE - GPS/TELEPHONIE - GPS_6855.jpg  
  inflating: images/TELEPHONIE - GPS/TELEPHONIE - GPS_6856.jpg  
  inflating: images/TELEPHONIE - GPS/TELEPHONIE - GPS_6857.jpg  
  inflating: images/TELEPHONIE - GPS/TELEPHONIE - GPS_6858.jpg  
  inflating: images/TELEPHONIE - GPS/TELEPHONIE - GPS_6859.jpg  
  inflating: images/TELEPHONIE - GPS/TELEPHONIE - GPS_686.jpg  
  inflating: i

In [ ]:
# Importez les bibliothèques nécessaires
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
# Chargement des données d'entraînement et de test
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [ ]:
# Séparez les données en entrées et étiquettes
X = train_data.iloc[:, 1:].values
y = train_data.iloc[:, 0].values


In [ ]:
# Effectuez un split des données d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Prétraitement des données
X_train = X_train.reshape(-1, 180, 180, 1)
X_val = X_val.reshape(-1, 180, 180, 1)
X_test = test_data.values.reshape(-1, 180, 180, 1)
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0
y_train = to_categorical(y_train, num_classes=5270)
y_val = to_categorical(y_val, num_classes=5270)

In [ ]:
# Génération de données supplémentaires
data_generator = ImageDataGenerator(rotation_range=10, zoom_range=0.1, width_shift_range=0.1, height_shift_range=0.1)
data_generator.fit(X_train)

In [ ]:
# Créez un modèle de réseau neuronal convolutionnel
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(180,180,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5270, activation='softmax'))

In [ ]:
# Compilez le modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Entraînez le modèle
model.fit_generator(data_generator.flow(X_train, y_train, batch_size=32), steps_per_epoch=len(X_train)/32, epochs=10, validation_data=(X_val, y_val))

In [ ]:
# Évaluez le modèle sur les données de validation
val_loss, val_acc = model.evaluate(X_val, y_val)
print("Validation Loss: ", val_loss)
print("Validation Accuracy: ", val_acc)

In [ ]:
# Utilisez le modèle pour prédire les catégories des données de test
test_predictions = model.predict(X_test)

In [ ]:
# Convertir les prédictions en catégories numériques
test_predictions = np.argmax(test_predictions, axis=1)

In [ ]:
# Enregistrer les prédictions dans un fichier CSV pour soumission
submission = pd.DataFrame({'Id': test_data.Id, 'Category': test_predictions})
submission.to_csv("submission.csv", index=False)

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop,SGD,Adam

In [11]:
TRAIN_DIR = "images/"
IMG_WIDTH = 180
IMG_HEIGHT = 180
BATCH_SIZE = 32

In [16]:
train_datagen = train_datagen = ImageDataGenerator( rescale = 1.0/255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, # same directory as training data
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation') # set as validation data

Found 67332 images belonging to 45 classes.
Found 16811 images belonging to 45 classes.


In [39]:
# Créez un modèle de réseau neuronal convolutionnel
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(180,180,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))

model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 89, 89, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_41 (MaxPoolin  (None, 43, 43, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_42 (Conv2D)          (None, 41, 41, 128)       73856     
                                                                 
 max_pooling2d_42 (MaxPoolin  (None, 20, 20, 128)    

In [41]:
# Compilez le modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entraînez le modèle
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=1200 // BATCH_SIZE,
                    epochs=1,
                    validation_steps=300 // BATCH_SIZE)

37/37 [==============================] - 81s 2s/step - loss: 0.0000e+00 - accuracy: 0.0042 - val_loss: 0.0000e+00 - val_accuracy: 0.0104
